# Lab 4 - Cortex Agents Setup

This notebook creates Snowflake Cortex Agents for the services built in Labs 1-3 and sets up an email agent for Snowflake Intelligence summaries.

## What We'll Create
- **LAB1_SEARCH_AGENT**: Agent for Lab 1 Cortex Search service (document search)
- **LAB2_ANALYST_AGENT**: Agent for Lab 2 Cortex Analyst service (wealth management analytics)
- **LAB3_SEARCH_AGENT**: Agent for Lab 3 Cortex Search service (multimodal document parsing)
- **Email Infrastructure**: Stored procedure and scheduled task for Snowflake Intelligence summaries

## Prerequisites
- Completed Labs 1, 2, and 3 with their respective Cortex services
- Appropriate privileges to create agents, procedures, and tasks
- Email notification integration configured (for email functionality)

## References
- [Cortex Agents Documentation](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-agents)
- [Snowflake Intelligence Overview](https://docs.snowflake.com/en/user-guide/snowflake-cortex/snowflake-intelligence)
- [Snowflake Intelligence Tools](https://github.com/snowflake-procs-tg/snowflake-intelligence-tools/tree/main)


## 1. Configuration

First, let's set up our configuration and discover existing services from the previous labs.


In [ ]:
-- Configuration - Update these values as needed
SET WAREHOUSE = 'CORTEX_SEARCH_TUTORIAL_WH';
SET ROLE = 'SYSADMIN';

-- Lab 1 - Search Service Configuration
SET LAB1_DATABASE = 'CORTEX_SEARCH_TUTORIAL_DB';
SET LAB1_SCHEMA = 'PUBLIC';
SET LAB1_SEARCH_SERVICE = 'DOCUMENT_SEARCH_SERVICE';

-- Lab 2 - Analyst Service Configuration
SET LAB2_DATABASE = 'CORTEX_ANALYST_DEMO';
SET LAB2_SCHEMA = 'WEALTH_MANAGEMENT';
SET LAB2_ANALYST_SERVICE = 'WEALTH_ANALYST_SERVICE';

-- Lab 3 - Search Service Configuration
SET LAB3_DATABASE = 'CORTEX_SEARCH_TUTORIAL_DB';
SET LAB3_SCHEMA = 'PUBLIC';
SET LAB3_SEARCH_SERVICE = 'DOCS_SEARCH_SERVICE';

-- Email Configuration (optional)
SET EMAIL_INTEGRATION = 'YOUR_EMAIL_INTEGRATION';  -- Update with your integration name
SET EMAIL_RECIPIENT = 'recipient@example.com';     -- Update with recipient email
SET EMAIL_SCHEDULE = 'USING CRON 0 9 * * * UTC';   -- Daily at 9 AM UTC

SELECT 'Configuration loaded successfully!' as status,
       'Warehouse: ' || $WAREHOUSE as warehouse_config,
       'Role: ' || $ROLE as role_config;


## 2. Environment Setup

Set up the Snowflake environment and verify our services exist.


In [ ]:
-- Set up environment
USE ROLE IDENTIFIER($ROLE);
USE WAREHOUSE IDENTIFIER($WAREHOUSE);

SELECT 'Environment setup complete' as status;


## 3. Create Cortex Agents

Now let's create the Cortex Agents for each of our services. Agents provide a unified interface to interact with Cortex services through Snowflake Intelligence.

### 3.1 Lab 1 Search Agent

This agent will handle document search queries from Lab 1's Cortex Search service.


In [ ]:
-- Create Lab 1 Search Agent
-- First build the full service name
SET FULL_SERVICE_NAME_LAB1 = $LAB1_DATABASE || '.' || $LAB1_SCHEMA || '.' || $LAB1_SEARCH_SERVICE;

CREATE OR REPLACE AGENT LAB1_SEARCH_AGENT (
  CORTEX_SEARCH_SERVICE = IDENTIFIER($FULL_SERVICE_NAME_LAB1)
);

SELECT 'Lab 1 Search Agent created successfully' as status;


### 3.2 Lab 2 Analyst Agent

This agent will handle wealth management analytics queries using the Cortex Analyst service from Lab 2.


In [ ]:
-- Create Lab 2 Analyst Agent
-- First build the full service name
SET FULL_SERVICE_NAME_LAB2 = $LAB2_DATABASE || '.' || $LAB2_SCHEMA || '.' || $LAB2_ANALYST_SERVICE;

CREATE OR REPLACE AGENT LAB2_ANALYST_AGENT (
  CORTEX_ANALYST_SERVICE = IDENTIFIER($FULL_SERVICE_NAME_LAB2)
);

SELECT 'Lab 2 Analyst Agent created successfully' as status;


### 3.3 Lab 3 Search Agent

This agent will handle multimodal document parsing queries from Lab 3's enhanced Cortex Search service.


In [ ]:
-- Create Lab 3 Search Agent
-- First build the full service name
SET FULL_SERVICE_NAME_LAB3 = $LAB3_DATABASE || '.' || $LAB3_SCHEMA || '.' || $LAB3_SEARCH_SERVICE;

CREATE OR REPLACE AGENT LAB3_SEARCH_AGENT (
  CORTEX_SEARCH_SERVICE = IDENTIFIER($FULL_SERVICE_NAME_LAB3)
);

SELECT 'Lab 3 Search Agent created successfully' as status;


## 4. Verify Agent Creation

Let's verify that all our agents were created successfully.


In [ ]:
-- Show all agents we just created
SHOW AGENTS LIKE '%LAB%_AGENT';


## 5. Email Infrastructure for Snowflake Intelligence

Now let's set up the email infrastructure to send periodic summaries of Snowflake Intelligence analyses.

### 5.1 Create Email Stored Procedure

This procedure will gather analysis summaries and send them via email.


In [ ]:
-- Create stored procedure for sending analysis emails
CREATE OR REPLACE PROCEDURE SEND_ANALYSIS_EMAIL(
    EMAIL_INTEGRATION STRING,
    RECIPIENT STRING
)
RETURNS STRING
LANGUAGE SQL
AS
$$
DECLARE
    analysis_summary STRING;
BEGIN
    -- Gather analysis summary from recent Cortex activity
    SELECT 
        'Snowflake Intelligence Summary for ' || CURRENT_DATE() || ':\n\n' ||
        'Total Cortex queries today: ' || COUNT(*) || '\n' ||
        'Unique users: ' || COUNT(DISTINCT USER_NAME) || '\n' ||
        'Most active database: ' || COALESCE(MODE(DATABASE_NAME), 'N/A') || '\n\n' ||
        'Lab Agents Activity:\n' ||
        '- LAB1_SEARCH_AGENT: Document search queries\n' ||
        '- LAB2_ANALYST_AGENT: Wealth management analytics\n' ||
        '- LAB3_SEARCH_AGENT: Multimodal document parsing\n\n' ||
        'Generated at: ' || CURRENT_TIMESTAMP()
    INTO analysis_summary
    FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
    WHERE START_TIME >= CURRENT_DATE()
    AND (QUERY_TEXT ILIKE '%CORTEX%' OR QUERY_TEXT ILIKE '%AGENT%');
    
    -- Send the email
    CALL SYSTEM$SEND_EMAIL(
        EMAIL_INTEGRATION,
        RECIPIENT,
        'Snowflake Intelligence Daily Summary - ' || CURRENT_DATE(),
        COALESCE(analysis_summary, 'No analysis data available for today.')
    );
    
    RETURN 'Email sent successfully at ' || CURRENT_TIMESTAMP();
EXCEPTION
    WHEN OTHER THEN
        RETURN 'Error sending email: ' || SQLERRM;
END;
$$;

SELECT 'Email procedure created successfully' as status;


### 5.2 Create Scheduled Task

This task will automatically run the email procedure on a schedule.


In [ ]:
-- Create scheduled task for email reports
-- Note: Update EMAIL_INTEGRATION and EMAIL_RECIPIENT in the config section before enabling
CREATE OR REPLACE TASK EMAIL_ANALYSIS_TASK
    WAREHOUSE = IDENTIFIER($WAREHOUSE)
    SCHEDULE = $EMAIL_SCHEDULE
AS
    CALL SEND_ANALYSIS_EMAIL($EMAIL_INTEGRATION, $EMAIL_RECIPIENT);

SELECT 'Email task created successfully (suspended by default)' as status;


### 5.3 Enable the Email Task (Optional)

Tasks are created in a suspended state by default. Run this cell when you're ready to activate automated emails.


In [ ]:
-- Enable the email task (uncomment when ready)
-- ALTER TASK EMAIL_ANALYSIS_TASK RESUME;

SELECT 'Email task enable cell ready (uncomment to activate)' as status;


## 6. Testing and Validation

Let's test our agents and email functionality.

### 6.1 Test Email Procedure (Optional)

**Note**: Only run this if you have configured a valid email integration.


In [ ]:
-- Test the email procedure (uncomment and update parameters if you want to test)
-- CALL SEND_ANALYSIS_EMAIL('YOUR_EMAIL_INTEGRATION', 'recipient@example.com');

SELECT 'Email test cell ready (uncomment to execute)' as status;


### 6.2 Monitor Task Status


In [ ]:
-- Check task status
SHOW TASKS LIKE 'EMAIL_ANALYSIS_TASK';


## 7. Usage in Snowflake Intelligence

🎉 **Congratulations!** Your agents are now ready to use in Snowflake Intelligence.

### How to Use Your Agents:

1. **Navigate to Snowflake Intelligence** in your Snowflake interface
2. **Select an agent** from the dropdown:
   - `LAB1_SEARCH_AGENT`: Document search from Lab 1
   - `LAB2_ANALYST_AGENT`: Wealth management analytics from Lab 2  
   - `LAB3_SEARCH_AGENT`: Multimodal document parsing from Lab 3

3. **Ask natural language questions** like:
   - For LAB1_SEARCH_AGENT: \"Find documents about pension considerations\"
   - For LAB2_ANALYST_AGENT: \"What's the total portfolio value by client segment?\"
   - For LAB3_SEARCH_AGENT: \"Analyze the 2023 factbook for investment trends\"

### Email Summaries:
- Configure your email integration and recipient in the configuration section
- Enable the task to receive daily summaries of Snowflake Intelligence activity
- Customize the email content by modifying the `SEND_ANALYSIS_EMAIL` procedure


## 8. Comprehensive Testing & Validation

Let's systematically test that everything works correctly before using the agents in production.

### 8.1 Prerequisites Check

First, verify that all the services from Labs 1-3 actually exist.


In [ ]:
-- Test 1: Verify Lab 1 Search Service exists
SELECT 'Testing Lab 1 Search Service...' as test_name;

SHOW CORTEX SEARCH SERVICES IN SCHEMA IDENTIFIER($LAB1_DATABASE || '.' || $LAB1_SCHEMA);

-- Check if our specific service exists
SELECT 
    CASE 
        WHEN EXISTS (
            SELECT 1 FROM TABLE(RESULT_SCAN(LAST_QUERY_ID())) 
            WHERE "name" = $LAB1_SEARCH_SERVICE
        ) 
        THEN '✅ Lab 1 Search Service found: ' || $LAB1_SEARCH_SERVICE
        ELSE '❌ Lab 1 Search Service NOT found: ' || $LAB1_SEARCH_SERVICE
    END as lab1_status;


In [ ]:
-- Test 2: Verify Lab 3 Search Service exists
SELECT 'Testing Lab 3 Search Service...' as test_name;

SHOW CORTEX SEARCH SERVICES IN SCHEMA IDENTIFIER($LAB3_DATABASE || '.' || $LAB3_SCHEMA);

SELECT 
    CASE 
        WHEN EXISTS (
            SELECT 1 FROM TABLE(RESULT_SCAN(LAST_QUERY_ID())) 
            WHERE "name" = $LAB3_SEARCH_SERVICE
        ) 
        THEN '✅ Lab 3 Search Service found: ' || $LAB3_SEARCH_SERVICE
        ELSE '❌ Lab 3 Search Service NOT found: ' || $LAB3_SEARCH_SERVICE
    END as lab3_status;


In [ ]:
-- Test 3: Verify Lab 2 Analyst components exist
SELECT 'Testing Lab 2 Analyst components...' as test_name;

-- Check if the database and schema exist
SELECT 
    CASE 
        WHEN EXISTS (SELECT 1 FROM INFORMATION_SCHEMA.SCHEMATA 
                    WHERE CATALOG_NAME = $LAB2_DATABASE 
                    AND SCHEMA_NAME = $LAB2_SCHEMA)
        THEN '✅ Lab 2 Database/Schema found: ' || $LAB2_DATABASE || '.' || $LAB2_SCHEMA
        ELSE '❌ Lab 2 Database/Schema NOT found: ' || $LAB2_DATABASE || '.' || $LAB2_SCHEMA
    END as lab2_schema_status;


### 8.2 Agent Creation Verification

Now let's test that our agents were created correctly and have the right configurations.


In [ ]:
-- Test 4: Verify all agents were created
SELECT 'Testing Agent Creation...' as test_name;

SHOW AGENTS LIKE '%LAB%_AGENT';

-- Get detailed agent information
SELECT 
    "name" as agent_name,
    "service_type",
    "database_name",
    "schema_name",
    "created_on",
    "owner",
    CASE 
        WHEN "name" IN ('LAB1_SEARCH_AGENT', 'LAB2_ANALYST_AGENT', 'LAB3_SEARCH_AGENT')
        THEN '✅ Expected agent found'
        ELSE '⚠️ Unexpected agent'
    END as status
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
ORDER BY "name";


In [ ]:
-- Test 5: Check agent count and validate expected agents exist
WITH expected_agents AS (
    SELECT agent_name FROM VALUES 
    ('LAB1_SEARCH_AGENT'),
    ('LAB2_ANALYST_AGENT'), 
    ('LAB3_SEARCH_AGENT')
    AS t(agent_name)
),
actual_agents AS (
    SELECT "name" as agent_name FROM TABLE(RESULT_SCAN(LAST_QUERY_ID(-1)))
    WHERE "name" LIKE '%LAB%_AGENT'
)
SELECT 
    '🧪 Agent Validation Summary' as test_summary,
    (SELECT COUNT(*) FROM expected_agents) as expected_count,
    (SELECT COUNT(*) FROM actual_agents) as actual_count,
    CASE 
        WHEN (SELECT COUNT(*) FROM expected_agents) = (SELECT COUNT(*) FROM actual_agents)
        THEN '✅ All expected agents created'
        ELSE '❌ Agent count mismatch'
    END as validation_result;


### 8.3 Agent Functionality Tests

Let's test that each agent can actually be described and is properly configured.


In [ ]:
-- Test 6: Describe each agent to verify configuration
SELECT 'Testing Agent Descriptions...' as test_name;

-- Test Lab 1 Search Agent
DESCRIBE AGENT LAB1_SEARCH_AGENT;


In [ ]:
-- Test Lab 2 Analyst Agent
DESCRIBE AGENT LAB2_ANALYST_AGENT;


In [ ]:
-- Test Lab 3 Search Agent
DESCRIBE AGENT LAB3_SEARCH_AGENT;


### 8.4 Email Infrastructure Tests

Test the email stored procedure and task creation.


In [ ]:
-- Test 7: Verify email procedure exists and can be described
SELECT 'Testing Email Infrastructure...' as test_name;

-- Check if the procedure exists
SELECT 
    CASE 
        WHEN EXISTS (
            SELECT 1 FROM INFORMATION_SCHEMA.PROCEDURES 
            WHERE PROCEDURE_NAME = 'SEND_ANALYSIS_EMAIL'
        )
        THEN '✅ Email procedure SEND_ANALYSIS_EMAIL found'
        ELSE '❌ Email procedure SEND_ANALYSIS_EMAIL NOT found'
    END as procedure_status;

-- Describe the procedure
DESCRIBE PROCEDURE SEND_ANALYSIS_EMAIL(STRING, STRING);


In [ ]:
-- Test 8: Verify email task exists and check its configuration
SHOW TASKS LIKE 'EMAIL_ANALYSIS_TASK';

-- Check task details
SELECT 
    "name" as task_name,
    "state" as task_state,
    "schedule" as schedule,
    "warehouse" as warehouse,
    "created_on",
    CASE 
        WHEN "state" = 'suspended' 
        THEN '✅ Task created (suspended - ready to enable)'
        WHEN "state" = 'started'
        THEN '✅ Task created and running'
        ELSE '⚠️ Task in unexpected state: ' || "state"
    END as task_status
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
WHERE "name" = 'EMAIL_ANALYSIS_TASK';


### 8.5 Integration Tests (Safe Dry-Run)

Test that agents can be accessed and are ready for Snowflake Intelligence without actually executing searches.


In [ ]:
-- Test 9: Verify agents are accessible from current context
SELECT 'Testing Agent Accessibility...' as test_name;

-- Test if we can show agents (this verifies permissions and accessibility)
SHOW AGENTS;

-- Count accessible agents
SELECT 
    COUNT(*) as total_agents,
    COUNT(CASE WHEN "name" LIKE '%LAB%_AGENT' THEN 1 END) as lab_agents,
    CASE 
        WHEN COUNT(CASE WHEN "name" LIKE '%LAB%_AGENT' THEN 1 END) = 3
        THEN '✅ All 3 lab agents accessible'
        ELSE '⚠️ Expected 3 lab agents, found: ' || COUNT(CASE WHEN "name" LIKE '%LAB%_AGENT' THEN 1 END)
    END as accessibility_status
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));


### 8.6 Test Summary & Health Check

Generate a comprehensive test summary to quickly assess the health of your Lab 4 setup.


In [ ]:
-- Test 10: Comprehensive Health Check Summary
SELECT '🏥 LAB 4 HEALTH CHECK SUMMARY' as health_check_title;

-- Agent Health Check
WITH agent_check AS (
    SELECT COUNT(*) as agent_count
    FROM INFORMATION_SCHEMA.CORTEX_AGENTS
    WHERE AGENT_NAME LIKE '%LAB%_AGENT'
),
procedure_check AS (
    SELECT COUNT(*) as procedure_count
    FROM INFORMATION_SCHEMA.PROCEDURES 
    WHERE PROCEDURE_NAME = 'SEND_ANALYSIS_EMAIL'
),
task_check AS (
    SELECT COUNT(*) as task_count, 
           COUNT(CASE WHEN STATE = 'suspended' THEN 1 END) as suspended_tasks
    FROM INFORMATION_SCHEMA.TASKS
    WHERE TASK_NAME = 'EMAIL_ANALYSIS_TASK'
)
SELECT 
    '📊 Component Status:' as category,
    'Agents: ' || agent_count || '/3 expected' as agents_status,
    'Email Procedure: ' || CASE WHEN procedure_count = 1 THEN '✅ Created' ELSE '❌ Missing' END as procedure_status,
    'Email Task: ' || CASE WHEN task_count = 1 THEN '✅ Created (' || suspended_tasks || ' suspended)' ELSE '❌ Missing' END as task_status,
    CASE 
        WHEN agent_count = 3 AND procedure_count = 1 AND task_count = 1
        THEN '🎉 ALL SYSTEMS GO! Ready for Snowflake Intelligence'
        ELSE '⚠️ Some components missing - check individual tests above'
    END as overall_health
FROM agent_check, procedure_check, task_check;


### 8.7 Next Steps After Testing

Based on your test results, here's what to do next:

#### ✅ If All Tests Pass:
1. **Navigate to Snowflake Intelligence** in your UI
2. **Select one of your agents** from the dropdown
3. **Try sample queries**:
   - LAB1_SEARCH_AGENT: "Find documents about retirement planning"
   - LAB2_ANALYST_AGENT: "What's the portfolio performance by region?"
   - LAB3_SEARCH_AGENT: "What are the key investment trends in 2023?"

#### ❌ If Tests Fail:
1. **Check Prerequisites**: Ensure Labs 1-3 services exist and are accessible
2. **Verify Permissions**: Make sure you have rights to create agents and procedures
3. **Update Configuration**: Double-check service names in the configuration section
4. **Check Error Messages**: Review any error output from the test cells above

#### 📧 To Enable Email Summaries:
1. **Configure Email Integration**: Set up a notification integration in Snowflake
2. **Update Configuration**: Replace placeholder values in the configuration section
3. **Test Email**: Uncomment and run the email test cell
4. **Enable Task**: Uncomment the task enable command when ready

#### 🔍 Troubleshooting Tips:
- **Service Not Found**: Go back to the respective lab and verify the service was created
- **Permission Denied**: Check that your role has the necessary privileges
- **Agent Creation Failed**: Verify the database/schema/service names are correct
- **Email Issues**: Ensure your notification integration is properly configured
